In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import sys

In [3]:
path = r"C:\Program Files (x86)\chromedriver.exe"
resume = r"C:\Users\pleasework\Documents\Resumes\ResumeJun2020.pdf"

In [4]:
## Things to add:
## applying to multiple positions at once
## checking requirements
## keep track of jobs that are not easy apply (company website)

# Goals: Weed out jobs before switching tabs (requirements)

In [105]:
def apply(job, location, name, email, phone, resume):
    driver = webdriver.Chrome(path)
    #Link to indeed home
    driver.get("https://www.indeed.com/?from=gnav-jobsearch--jasx")
    jobtitle = driver.find_element_by_name("q")
    jobtitle.send_keys(job)
    where = driver.find_element_by_name("l")
    #Clears the default location on indeed
    #hacky solution - maxlen allowed for place is 62, so just press backspace 62 times lol
    #alternative solution - select all, then backspace in the field
    for _ in range(62):
        where.send_keys(Keys.BACKSPACE);
    where.send_keys(location)
    where.send_keys(Keys.RETURN)
    
    ####Applying####
    for i in range(4):
        time.sleep(2)
        # Prep page iterating. Both forward and backwards have the same class name
        next_page = driver.find_elements_by_class_name("np")
        page_clicker = ActionChains(driver)
        cards = driver.find_elements_by_class_name("jobsearch-SerpJobCard")
        job_dates = driver.find_elements_by_class_name("date")

        # New list of jobs if the job was posted less than 20 days ago, if "Today"/"Just posted" make it 0
        job_dates = [0 if (job.text[0:2] == "To" or job.text[0:2] =="Ju") else job.text[0:2] for job in job_dates]

        # Remove cards based on the indices where day posted was over 20 days ago
        indices = [index for index, job in enumerate(job_dates) if float(job) < 20] 
        new_cards = [cards[index] for index in indices]
        # Make new list of job postings where can apply through indeed
        easy_cards = [card for card in new_cards if "Easily apply" in card.get_attribute('innerHTML')]

        # If there are no recent and easy apply jobs, move onto next page            
        if not easy_cards:
            print("No new jobs")
            click_next(next_page, driver)
            continue
            
    for card in new_cards:
        # only click the ones with indeed easy apply
        if "Easily apply" in card.get_attribute('innerHTML'):
            link = card.find_element_by_xpath(".//a[@class='jobtitle turnstileLink ']")
            actions = ActionChains(driver)
            actions.key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()
            driver.switch_to.window(driver.window_handles[-1])
            apply = driver.find_elements_by_tag_name("button")
            apply[2].click()
            html = driver.page_source
            iframe = driver.find_elements_by_tag_name("iframe")
            framenamesrc = [(i.get_attribute('name'), i.get_attribute('src')) for i in iframe]
            for framename, src in framenamesrc:
                if framename.startswith('indeedapply'):
                    driver.switch_to.frame(framename)
                    iframe = driver.find_elements_by_tag_name("iframe")
                    iframeinner = driver.find_elements_by_tag_name("iframe")
                    srcinner = [i.get_attribute('src') for i in iframeinner]
                    for innersrc in srcinner:
                        if innersrc.startswith('https://apply.indeed.com/indeedapply/'):
                            print(innersrc)
                            driver.get(innersrc)
                            break
                    break
   #         time.sleep(3)
    #        try:
     #           namefield = driver.find_element_by_id("input-applicant.name")
      #          namefield.send_keys("")
       #         namefield.send_keys(name)
        #    except:
         #       print("Names are split into two fields")
          #  try:
           #     firstName, lastName = name.split(" ")
            #    firstName = driver.find_element_by_id("input-applicant.firstName")
             #   firstName.send_keys("")
    #            firstName.send_keys(firstName)
     #           lastName = driver.find_element_by_id("input-applicant.lastName")
      #          lastName.send_keys("")
       #         lastName.send_keys(lastName)
        #    except: 
         #       print("Now we have a problem")
          #  emailfield = driver.find_element_by_id("input-applicant.email")
           # emailfield.send_keys(email)
 #           phonefield = driver.find_element_by_id("input-applicant.phoneNumber")
  #          phonefield.send_keys(phone)
   #         resumefield = driver.find_element_by_id("ia-FilePicker-resume")
    #        resumefield.send_keys(resume)
     #       driver.switch_to.window(driver.window_handles[0])
    time.sleep(3)
#    driver.quit()

In [1]:
apply("Data Analyst", "Boston, Massachusetts", "Felix Chan", "thefelixchan@gmail.com", "9736195672", resume)

NameError: name 'apply' is not defined

In [4]:
# Clicks the next page element, regardless of the dismiss banner
def click_next(element_list, driver):
    # If the list is longer than one element, that means it will have to choose the second to advance
    if len(element_list) == 1:
        driver.execute_script("arguments[0].click();", element_list[0])
    else:
        driver.execute_script("arguments[0].click();", element_list[1])

In [5]:
def apply(job, location, name, email, phone, resume):
    driver = webdriver.Chrome(path)
    #Link to indeed home
    driver.get("https://www.indeed.com/?from=gnav-jobsearch--jasx")
    jobtitle = driver.find_element_by_name("q")
    jobtitle.send_keys(job)
    where = driver.find_element_by_name("l")
    wait = WebDriverWait(driver, 5)
    #Clears the default location on indeed
    #hacky solution - maxlen allowed for place is 62, so just press backspace 62 times lol
    #alternative solution - select all, then backspace in the field
    for _ in range(62):
        where.send_keys(Keys.BACKSPACE);
    where.send_keys(location)
    where.send_keys(Keys.RETURN)
    jobPage = driver.current_url
    ####Applying####
    next_step = ''
    while (next_step != "C" and next_step != "D"):
        time.sleep(2)
        # Prep page iterating. Both forward and backwards have the same class name
        next_page = driver.find_elements_by_class_name("np")
        page_clicker = ActionChains(driver)
        cards = driver.find_elements_by_class_name("jobsearch-SerpJobCard")
        job_dates = driver.find_elements_by_class_name("date")

        # New list of jobs if the job was posted less than 20 days ago, if "Today"/"Just posted" make it 0
        job_dates = [0 if (job.text[0:2] == "To" or job.text[0:2] =="Ju") else job.text[0:2] for job in job_dates]

        # Remove cards based on the indices where day posted was over 20 days ago
        indices = [index for index, job in enumerate(job_dates) if float(job) < 20] 
        new_cards = [cards[index] for index in indices]
        # Make new list of job postings where can apply through indeed
        easy_cards = [card for card in new_cards if "Easily apply" in card.get_attribute('innerHTML')]

        # If there are no recent and easy apply jobs, move onto next page            
        if not easy_cards:
            print("No new jobs")
            click_next(next_page, driver)
            jobPage = driver.current_url

            
        # For future iterations, keep a list of company names so reloading cards is easier(?)
        all_companies = []
        for card in easy_cards:
            all_companies.append(card.find_element_by_class_name("company").text)
            
        # Create a list to keep track of companies applied to, prevent multiple application attempts
        appliedTo = []
        # only click the ones with indeed easy apply
        for card in easy_cards:
            # Solution found for making presence of element a boolean
            if driver.find_elements_by_id("popover-close-link"):
                dismiss = driver.find_element_by_id("popover-close-link")
                dismiss.click()
                time.sleep(2)
                driver.switch_to.frame(0)

            # On new iterations, gets stale element
            try:
                link = card.find_element_by_css_selector('a').get_attribute('href')       
            except: #StaleElementReferenceException:
              #  waitLink = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "jobsearh-SerpJobCard")))
                driver.refresh()
                easy_cards = driver.find_elements_by_class_name("jobsearch-SerpJobCard")
                easy_cards = [card for card in easy_cards if card.find_element_by_class_name("company").text in all_companies]
                link = card.find_element_by_css_selector('a').get_attribute('href') 
            company = card.find_element_by_class_name("company").text

            if company in appliedTo:
                continue
            else:
                appliedTo.append(company)


                print("Company list", appliedTo)

            time.sleep(1)
            

            driver.execute_script('''window.open("{}","_blank");'''.format(link))
            driver.switch_to.window(driver.window_handles[1])
            html = driver.page_source
            # Check requirements. Some cases where both bachelors and masters are ok
            # Will check if bachelors isn't listed, if not, make sure others are unlisted too
            unwanted_keywords = ['master', 'ms', 'phd', 'ph.d.']
            page_text = html.lower()
            if ('bachelor' not in page_text) and ('bs' not in page_text):
                if any(keyword in page_text for keyword in unwanted_keywords):
                    if re.search(r'([3-9]+(?:-[4-9]+)?\+?)\s*(years?)', page_text):
                        continue

                        
            apply = driver.find_elements_by_tag_name("button")
            ## Occasionally get an issue where button apply[2] is out of range (not sure why)
            try:
                apply[2].click()
            except:
                apply[1].click()

            iframe = driver.find_elements_by_tag_name("iframe")
            framenamesrc = [(i.get_attribute('name'), i.get_attribute('src')) for i in iframe]
            for framename, src in framenamesrc:
                if framename.startswith('indeedapply'):
                    driver.switch_to.frame(framename)
                    iframe = driver.find_elements_by_tag_name("iframe")
                    iframeinner = driver.find_elements_by_tag_name("iframe")
                    srcinner = [i.get_attribute('src') for i in iframeinner]
                    for innersrc in srcinner:
                        if innersrc.startswith('https://apply.indeed.com/indeedapply/'):
                            print(innersrc)
                            driver.get(innersrc)
                            break
                    break
            try:
                namefield = driver.find_element_by_id("input-applicant.name")
                namefield.send_keys("")
                namefield.send_keys(name)
            except:
                firstName, lastName = name.split(" ")
                firstField = driver.find_element_by_id("input-applicant.firstName")
                firstField.send_keys("")
                firstField.send_keys(firstName)
                lastField = driver.find_element_by_id("input-applicant.lastName")
                lastField.send_keys("")
                lastField.send_keys(lastName)
            emailfield = driver.find_element_by_id("input-applicant.email")
            emailfield.send_keys(email)
            # Sometimes phone field is not available
            if(driver.find_elements_by_id("input-applicant.phoneNumber")):
                phonefield = driver.find_element_by_id("input-applicant.phoneNumber")
                phonefield.send_keys(phone)
            resumefield = driver.find_element_by_id("ia-FilePicker-resume")
            resumefield.send_keys(resume)
           # driver.switch_to.window(driver.window_handles[0])
            ### Make continuing rely on user input
            input_keywords = ["C", "D"]
            next_step = input("Press C to continue, D to be done ")
            next_step = next_step.upper()
            if(next_step == 'D'):
                driver.quit()
                sys.exit()
            elif(next_step == 'C'):
                next_step = ''
                driver.execute_script("window.close();")
                driver.switch_to.window(driver.window_handles[0])
                continue
        driver.quit()

In [8]:
apply("Data Analyst", "Boston, Massachusetts", "Felix Chan", "thefelixchan@gmail.com", "9736195672", resume)

Company list ['Medicem, Inc.']


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="input-applicant.firstName"]"}
  (Session info: chrome=83.0.4103.116)
